In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
use_gpu = torch.cuda.is_available()
import gensim

from tqdm import tqdm_notebook, trange

from sklearn.model_selection import train_test_split

In [2]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    if shuffle: 
        indices = np.random.permutation(len(inputs))
    for start_idx in trange(0, len(inputs) + 1, batchsize):
        #print(start_idx)
        #print(start_idx + batchsize)
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

Протестируем iterate_minibatches. Готовая функция из семинаров немного косячила (не выдавала последнюю часть размера input_len % batch_size)

In [3]:
XX = np.random.normal(size=(111, 5))
YY = np.random.normal(size=111)
for x_batch, y_batch in iterate_minibatches(XX, YY, 95):
    print(x_batch.shape, y_batch.shape)

100%|██████████| 2/2 [00:00<00:00, 9998.34it/s]

(95, 5) (95,)
(16, 5) (16,)


In [4]:
data = pd.read_csv('all_data_normalized.csv')
data.head()

/Users/ak/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,score,source,text,text_normalized,len
0,0,5,foursquare,"2 рабочих места, выделенное рабочее место по к...",рабочий место выделять рабочий место кредит касса,7
1,1,5,foursquare,"pos-терминал, сенсорный экран для управления ""...",pos терминал сенсорный экран для управление сб...,12
2,2,5,foursquare,очереди имеются.,очередь иметься,2
3,3,5,foursquare,"free wi-fi ""tattelecom_unlim"" :) good while wa...",free wi fi tattelecom good while waiting,7
4,4,5,foursquare,so pretty interior though,so pretty interior though,4


In [5]:
import sys
sys.path.append('/Users/ak/Yandex.Disk.localized/sentiment-neural_past_from_Denis_Kirjanov/src')

In [6]:
root_path = '/Users/ak/Yandex.Disk.localized/Магистратура ВШЭ/Chat_word_autofill/'
embedding_model = gensim.models.Word2Vec
embedding_model = embedding_model.load(root_path + 'model_normalized_with_chats_2.bin')

In [7]:
data = data[data['len'] > 0]

In [8]:
data.score.value_counts()

1    262789
5    124411
3    116753
2     52459
4     18975
0         3
Name: score, dtype: int64

Уберем из данных тексты с промежуточными оценками. Можно было бы и распределить по оставшимся, но это неоднозначное решение

In [9]:
data = data[data.score.isin([1, 3, 5])]

In [10]:
data.score.value_counts()

1    262789
5    124411
3    116753
Name: score, dtype: int64

In [11]:
data.dropna(inplace=True, subset=['text_normalized'])

In [12]:
data.sort_values(by=['len'], ascending=False, inplace=True)

Заполняем представления. Максимальную длину предложения выбираем таким образом, чтобы 99% предложений попали полностью. Остальные обрезаем

In [24]:
CUTOFF = int(1e4)
lengths = data['len'].values[:CUTOFF]

max_length = int(np.percentile(data['len'], 99))
embedding_size = embedding_model.wv.vector_size
unknown = np.ones(embedding_size).astype('float32') * 7
num_unknown = 0
embeddings = np.zeros([CUTOFF, max_length, embedding_size])
i = 0
for text in tqdm_notebook(data.text_normalized.values[:CUTOFF]):
    #emb = np.zeros([embedding_size, max_length]).astype('float32')
    words = text.split()
    if lengths[i] > max_length:
        lengths[i] = max_length
    for j in range(min(len(words), max_length)):
        try:
            #emb[:, i] = embedding_model.wv[words[i]]#.astype('float32')
            embeddings[i, j, :] = embedding_model.wv[words[j]]
        except KeyError:
            embeddings[i, j, :] = unknown
            num_unknown += 1
            #print(words[i])
    i += 1
print('Неизвестных слов:', num_unknown)

HBox(children=(IntProgress(value=0, max=10000), HTML(value='')))


Неизвестных слов: 23389


In [36]:
scores = data.score.values[:CUTOFF]

In [28]:
packed = pack_padded_sequence(Variable(torch.from_numpy(embeddings)), lengths, batch_first=True)

In [38]:
packed

PackedSequence(data=Variable containing:
 7.0000e+00  7.0000e+00  7.0000e+00  ...   7.0000e+00  7.0000e+00  7.0000e+00
 7.0000e+00  7.0000e+00  7.0000e+00  ...   7.0000e+00  7.0000e+00  7.0000e+00
 1.7459e-01 -1.2062e+00  3.0000e-01  ...  -1.0233e+00  5.0580e-01 -9.9476e-01
                ...                   ⋱                   ...                
-1.0303e+00 -4.5521e-01 -1.0325e+00  ...  -1.1374e+00 -3.6873e-02 -1.8809e+00
-3.4304e-02 -2.2295e-02  6.7112e-01  ...   2.7297e-01 -2.8302e-01  1.4054e+00
 4.7893e-01 -2.8866e+00  6.3675e-01  ...   6.6361e-01 -3.3833e-01 -9.0275e-01
[torch.DoubleTensor of size 302672x300]
, batch_sizes=[10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 9818, 8499, 7436, 6433, 5576, 4910])

In [26]:
len(lengths), embeddings.shape, max(lengths)

(10000, (10000, 32, 300), 32)

In [30]:
a = torch.Tensor([[12,13,14,15], [2,6,9,3], [2,3,0,0], [1,4,0,0]])
xx = pack_padded_sequence(a, [4, 2], batch_first=False)
xx

ValueError: lengths array has incorrect size

Нужно разобраться с pack_padded_sequence

In [105]:
np.random.seed(100500)
emb_size = 12
vec1 = np.random.normal(size=[emb_size,5], loc=10, scale=1) * 100
vec2 = np.hstack((np.random.uniform(size=[emb_size, 3]) * 50, np.zeros([emb_size, 2])))
vec3 = np.hstack([np.random.uniform(size=[emb_size, 2]), np.zeros([emb_size, 3])])
vec = np.dstack([vec1, vec2, vec3])
print(vec)

[[[9.19344174e+02 2.74072352e+01 6.38354858e-02]
  [1.10168698e+03 7.24852318e-01 9.56330245e-01]
  [9.75431071e+02 2.64327000e+01 0.00000000e+00]
  [1.10624397e+03 0.00000000e+00 0.00000000e+00]
  [1.06370142e+03 0.00000000e+00 0.00000000e+00]]

 [[1.11646582e+03 3.51659927e+00 9.01379721e-01]
  [9.18423441e+02 4.60765079e+01 1.00025268e-01]
  [8.67228755e+02 2.25143125e+01 0.00000000e+00]
  [1.08703853e+03 0.00000000e+00 0.00000000e+00]
  [1.05876788e+03 0.00000000e+00 0.00000000e+00]]

 [[1.05934238e+03 3.04389368e+00 1.41118283e-01]
  [8.01125879e+02 3.99937466e+01 6.96427843e-01]
  [1.04267375e+03 3.62776941e+01 0.00000000e+00]
  [8.70328594e+02 0.00000000e+00 0.00000000e+00]
  [9.08048982e+02 0.00000000e+00 0.00000000e+00]]

 [[1.04314573e+03 2.11993418e+01 6.43678521e-01]
  [9.82347960e+02 1.96481519e+01 7.49106319e-02]
  [1.03636169e+03 1.17672199e+00 0.00000000e+00]
  [1.05779951e+03 0.00000000e+00 0.00000000e+00]
  [1.17490803e+03 0.00000000e+00 0.00000000e+00]]

 [[1.1469804

In [106]:
vec.shape

(12, 5, 3)

In [107]:
a = torch.from_numpy(vec.transpose(1, 2, 0))
a.shape

torch.Size([5, 3, 12])

In [108]:
a[0, :, :]



Columns 0 to 5 
 9.1934e+02  1.1165e+03  1.0593e+03  1.0431e+03  1.1470e+03  9.2893e+02
 2.7407e+01  3.5166e+00  3.0439e+00  2.1199e+01  2.0307e+01  3.9086e+01
 6.3835e-02  9.0138e-01  1.4112e-01  6.4368e-01  4.4368e-01  3.0282e-01

Columns 6 to 11 
 1.0405e+03  9.0487e+02  9.9522e+02  1.0983e+03  1.1457e+03  1.0180e+03
 2.3688e+01  3.1756e+01  2.5450e+01  1.6613e+01  2.5540e+01  1.0495e+01
 8.5236e-01  8.5344e-01  6.2959e-01  8.2850e-01  7.4975e-01  9.4671e-01
[torch.DoubleTensor of size 3x12]

In [109]:
xx = pack_padded_sequence(a, [5, 3, 2], batch_first=False)

In [110]:
xx

PackedSequence(data=

Columns 0 to 5 
 9.1934e+02  1.1165e+03  1.0593e+03  1.0431e+03  1.1470e+03  9.2893e+02
 2.7407e+01  3.5166e+00  3.0439e+00  2.1199e+01  2.0307e+01  3.9086e+01
 6.3835e-02  9.0138e-01  1.4112e-01  6.4368e-01  4.4368e-01  3.0282e-01
 1.1017e+03  9.1842e+02  8.0113e+02  9.8235e+02  9.6650e+02  9.8272e+02
 7.2485e-01  4.6077e+01  3.9994e+01  1.9648e+01  3.8730e+01  4.2484e+01
 9.5633e-01  1.0003e-01  6.9643e-01  7.4911e-02  4.4642e-01  8.1519e-01
 9.7543e+02  8.6723e+02  1.0427e+03  1.0364e+03  1.0344e+03  1.0583e+03
 2.6433e+01  2.2514e+01  3.6278e+01  1.1767e+00  1.1741e+01  2.2346e+01
 1.1062e+03  1.0870e+03  8.7033e+02  1.0578e+03  7.8497e+02  9.5498e+02
 1.0637e+03  1.0588e+03  9.0805e+02  1.1749e+03  1.0151e+03  1.1670e+03

Columns 6 to 11 
 1.0405e+03  9.0487e+02  9.9522e+02  1.0983e+03  1.1457e+03  1.0180e+03
 2.3688e+01  3.1756e+01  2.5450e+01  1.6613e+01  2.5540e+01  1.0495e+01
 8.5236e-01  8.5344e-01  6.2959e-01  8.2850e-01  7.4975e-01  9.4671e-01
 1.2696e

In [19]:
class SentimentRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers, cell='rnn'):
        super(SentimentRNN, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.cell = cell
        if cell == "rnn":
            self.rnn = nn.RNN(hidden_size, hidden_size, n_layers)
        else:
            self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        bs = input.size(0)
        reordered = input.view(1, bs, -1)
        output, hidden = self.rnn(reordered, hidden)
        return output, hidden
    
    def init_hidden(self, batch_size):
        hidden = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))
        if self.cell == "lstm":
            c0 = Variable(torch.randn(self.n_layers, batch_size, self.hidden_size))
            return (hidden, c0)
        return hidden

In [35]:
n_layers = 5
batch_size = 256
model = SentimentRNN(250, 3, n_layers, 'rnn')
if use_gpu:
    model = model.cuda()

Функция, преобразующая список предложений в word2vec 

In [169]:
def embed(sentences, embedder):
    l = len(sentences[0].split())
    n = embedder.wv.vector_size
    bs = len(sentences)
    res = np.zeros([l, bs, n])
    for i in range(bs):
        words = sentences[i].split()
        for j in range(l):
            res[j, i, :] = embedder.wv[words[j]]
    return res

Поскольку разобраться с pack_padded_sequences пока не удалось, то будем подавать батчи из предложений одинаковой длины, начиная с самых длинных

In [ ]:
def train_epoch(model, optimizer, inputs, targets):
    loss_log = []
    model.train()
    for X, y in iterate_minibatches(inputs, targets):
        x_batch = embed(X, embedding_model)
        hidden = model.init_hidden(x_batch.shape[0])
        optimizer.zero_grad()
        loss = 0.0
        for i in range(0, t.shape[1] - 1):
            out, hidden = model.forward(t[:, i], hidden)
            loss += F.cross_entropy(out, t[:, i+1])
        loss.backward()
        optimizer.step()
        
        loss = loss.data[0]
        loss_log.append(loss)
    return loss_log   

def test(model, x_batch, y_batch):
    loss_log = []
    model.eval()
    for batch in test_batches:
        nums = to_matrix(batch)
        t = Variable(torch.from_numpy(nums))
        y = Variable(torch.from_numpy(nums[:,1:]))
        x = Variable(torch.from_numpy(nums[:, :-1]))
        hidden = model.init_hidden(x.shape[0])
        loss = 0.0
        for i in range(0, t.shape[1] - 1):
            out, hidden = model.forward(t[:, i], hidden)
            loss += F.cross_entropy(out, t[:, i+1])

        loss = loss.data[0]
        loss_log.append(loss)
    return loss_log

def plot_history(train_history, val_history, title='loss'):
    plt.figure()
    plt.title('{}'.format(title))
    plt.plot(train_history, label='train', zorder=1)    
    points = np.array(val_history)

    plt.scatter(points[:, 0], points[:, 1], marker='+', s=180, c='orange', label='val', zorder=2)

    plt.xlabel('train steps')
    plt.legend(loc='best')
    plt.grid()
    plt.show()
    
def train(model, opt, n_epochs):
    unique_lengths = np.unique([min(x, max_length) for x in data['len'].unique()])
    #lengths = data['len'].values
    # Индексы, в которых появляются новые длины
    #change_indices = np.where(np.diff(lengths) < 0)[0] + 1
    
    train_log = []
    val_log = []
    
    for epoch in range(n_epochs):
        for ul in unique_lengths:
            if ul >= max_length:
                data_cur = data[data['len'] >= ul]
            else:
                data_cur = data[data['len'] == ul]
            X = data_cur.text_normalized.values
            y = data_cur.score.values
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=234769)
            steps = X_train.shape[0] / batch_size
                
            train_loss = train_epoch(model, opt, X_train, y_train)
            train_log.extend(train_loss)

            val_loss = test(model, X_test, y_test)
            val_log.append((steps * (epoch + 1), np.mean(val_loss)))
            clear_output()
            plot_history(train_log, val_log)

In [115]:
a = np.array([3,3,3,2,2,1,1,1,1])
np.where(np.diff(a) < 0)[0] + 1

array([3, 5])

In [126]:
a = np.concatenate([np.ones(16)*3, np.ones(12)*2, np.ones(8)])
indices = [0] + list(np.where(np.diff(a) < 0)[0] + 1) + [len(a)]

In [127]:
batch_size = 5
for i in range(len(indices) - 1):
    print(np.array_split(a[indices[i]:indices[i + 1]], (indices[i + 1] - indices[i] + 1) // batch_size)) 

[array([3., 3., 3., 3., 3., 3.]), array([3., 3., 3., 3., 3.]), array([3., 3., 3., 3., 3.])]
[array([2., 2., 2., 2., 2., 2.]), array([2., 2., 2., 2., 2., 2.])]
[array([1., 1., 1., 1., 1., 1., 1., 1.])]


In [141]:
from itertools import groupby, zip_longest

In [143]:
ind = list(range(len(a)))
for k, g in groupby(a):
    print(k)
    #print(list(g))
    for x in grouper(g, 5):
        print(x)

3.0
(3.0, 3.0, 3.0, 3.0, 3.0)
(3.0, 3.0, 3.0, 3.0, 3.0)
(3.0, 3.0, 3.0, 3.0, 3.0)
(3.0, None, None, None, None)
2.0
(2.0, 2.0, 2.0, 2.0, 2.0)
(2.0, 2.0, 2.0, 2.0, 2.0)
(2.0, 2.0, None, None, None)
1.0
(1.0, 1.0, 1.0, 1.0, 1.0)
(1.0, 1.0, 1.0, None, None)


In [138]:
# Функция, позволяющая в цикле перебирать подмассивы по n значений вместо отдельных элементов
def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [ ]:
%%time
opt = torch.optim.Adam(model.parameters(), lr=1e-2)
train(model, opt, 5)

In [144]:
data['len'].unique()

array([619, 615, 611, 610, 608, 604, 601, 599, 598, 595, 591, 590, 589,
       588, 585, 583, 582, 576, 575, 574, 572, 571, 568, 563, 330, 319,
       216, 200, 182, 181, 176, 159, 155, 151, 150, 142, 141, 136, 132,
       130, 129, 125, 123, 122, 121, 116, 115, 114, 112, 111, 110, 108,
       107, 105, 103, 101,  98,  97,  95,  94,  93,  92,  91,  90,  89,
        88,  87,  86,  85,  84,  83,  82,  81,  80,  79,  78,  77,  76,
        75,  74,  73,  72,  71,  70,  69,  68,  67,  66,  65,  64,  63,
        62,  61,  60,  59,  58,  57,  56,  55,  54,  53,  52,  51,  50,
        49,  48,  47,  46,  45,  44,  43,  42,  41,  40,  39,  38,  37,
        36,  35,  34,  33,  32,  31,  30,  29,  28,  27,  26,  25,  24,
        23,  22,  21,  20,  19,  18,  17,  16,  15,  14,  13,  12,  11,
        10,   9,   8,   7,   6,   5,   4,   3,   2,   1])